# Автоматизация обработки финансовой отчетности от крупной региональной аптечной сети

**Описание проекта**

Региональная аптечная сеть, состоящая из около 100 аптек, «переезжает» с упрощенной системы налогообложения на общую. 

**Описание задачи**

Необходимо произвести единоразовую загрузку файла в 1С, но для этого надо сделать сравнение реализации по аптекам и выгрузки из СБИСа. Пункты, которые расходятся, далее будут подправлены вручную и загружены в 1С. Подробная информация ниже.

**Входные данные:**

* Файлы с выгрузкой из СБИСа находся в папке 'Входящие'
* Файлы 'csv' с выгрузкой из аптек находятся в папке 'Аптеки/csv/correct/'

**Ход работы**

* Выгрузка из СБИС находится в папке Входящие. Загружаем все csv-файлы в один датафрейм. Если встречается файл с другим расширением, то его пропускаем. Нужно быть быть аккуратными с заголовками столбцов и не мешать их с данными при объединении нескольких файлов.


* Дадим столбцам полученного датафрейма такие названия:
    1. "Дата",
    2. "Номер",
    3. "Сумма",
    4. "Статус",
    5. "Примечание",
    6. "Комментарий",
    7. "Контрагент",
    8. "ИНН/КПП",
    9. "Организация",
    10. "ИНН/КПП",
    11. "Тип документа",
    12. "Имя файла",
    13. "Дата",
    14. "Номер 1",
    15. "Сумма 1",
    16. "Сумма НДС",
    17. "Ответственный",
    18. "Подразделение",
    19. "Код",
    20. "Дата",
    21. "Время",
    22. "Тип пакета",
    23. "Идентификатор пакета",
    24. "Запущено в обработку",
    25. "Получено контрагентом",
    26. "Завершено",
    27. "Увеличение суммы",
    28. "НДC",
    29. "Уменьшение суммы",
    30. "НДС"


* Все названия из нескольких слов сцепляем через нижнее подчеркивание в одно_длинное_слово.


* Файлы с выгрузкой из аптек находятся в папке 'Аптеки/csv/correct/'.


* Загружаем на обработку по одному файлу. Содержимое каждого файла загружаем в датафрейм Pandas. Если встречается не csv-формат — игнорируем его.


* Создаем столбцы:
    1. "Номер счет-фактуры"
    2. "Сумма счет-фактуры"
    3. "Дата счет-фактуры"
    4. "Сравнение дат"


* В каждой строке проверим:
    1. Если "Поставщик" — ЕАПТЕКА, то к "Номер накладной" нужно добавить '/15'.
    2. Нужно найти все записи в выгрузке из СБИСа по данному номеру накладной.
    3. Из найденных строк нужно оставить только те, которые имеют один из типов документа: ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"].
    4. Если ничего не найдено — просто переходим к следующей строке.
    5. Если найдено — нужно сохранить значения Номер, Сумма и Дата.
    6. Дату нужно представить в формате 25.05.2021.
    7. В столбцы из пункта 6 нужно записать найденные для данной строки значения.
    8. В столбец 'Сравнение дат' помещаем "Не совпадает!", если найденная дата и дата накладной отличаются. Иначе — пустая строка.


* Итоговый файл выгрузки по аптеке должен содержать такие столбцы в заданном порядке:\
'№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',\
'Дата приходного документа', 'Номер приходного документа',\
'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',\
'Сумма счет-фактуры', 'Кол-во',\
'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',\
'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат'


* Файл сохраняем по пути Результат/{полная сегодняшняя дата}/{имя исходного файла без расширения} — результат.xlsx. Если таких папок не существует — создадим их (проверку и создание нужно реализовать средствами Python, автоматически).

In [1]:
import os

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob

import warnings

In [153]:
# Чтобы часть столбцов в дальнейшем не скрывалась, настроим принудительное отображение
pd.set_option('display.max_columns', None)
# Настроим ширину столбцов
pd.set_option('display.max_colwidth', 999)
# Настроим отображение чисел с точностью до двух знаков после запятой (до сотых)
# 3.514284e+05 — научный формат вывода чисел. Означает 3.51 * 105
# pd.set_option('display.float_format', '{:.2f}'.format)
# pd.set_option.display.float_format = '{:.2f}'.format

In [154]:
warnings.simplefilter(action='ignore', category=Warning)

In [155]:
result_path = f'Результат/{datetime.now().strftime("%Y-%m-%d")}/'
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [156]:
sbis_names = ["Дата",
            "Номер",
            "Сумма",
            "Статус",
            "Примечание",
            "Комментарий",
            "Контрагент",
            "ИНН/КПП",
            "Организация",
            "ИНН/КПП",
            "Тип документа",
            "Имя файла",
            "Дата",
            "Номер 1",
            "Сумма 1",
            "Сумма НДС",
            "Ответственный",
            "Подразделение",
            "Код",
            "Дата",
            "Время",
            "Тип пакета",
            "Идентификатор пакета",
            "Запущено в обработку",
            "Получено контрагентом",
            "Завершено",
            "Увеличение суммы",
            "НДC",
            "Уменьшение суммы",
            "НДС"
            ]

In [12]:
# Определяем путь нахождения файлов с выгрузкой из аптек — папка 'Входящие'
sbis_folder = r'C:\Users\1C\Входящие'
sbis_files = os.listdir(sbis_folder)
sbis_files

['Входящие 01.csv',
 'Входящие 01.xlsx',
 'Входящие 02.csv',
 'Входящие 02.xlsx',
 'Входящие 03.csv',
 'Входящие 03.xlsx']

In [158]:
# Загружаем все csv-файлы в один датафрейм. 
# Если встречается файл с другим расширением, пропускаем его
dfs = []
for file in sbis_files:
    if 'csv' not in file:
        continue
    df = pd.read_csv(sbis_folder + "/" + file, skiprows=1, sep = ";", encoding="windows-1251", header=None)
    dfs.append(df)
dfs

[             0            1          2                             3   4   \
 0      30.09.21  БРН00709545   4 056,46  Выполнение завершено успешно NaN   
 1      30.09.21  БРН00709545   4 056,46  Выполнение завершено успешно NaN   
 2      30.09.21  БРН00709545       0,00  Выполнение завершено успешно NaN   
 3      30.09.21  БРН00709520  54 705,65  Выполнение завершено успешно NaN   
 4      30.09.21  БРН00709520  54 705,65  Выполнение завершено успешно NaN   
 ...         ...          ...        ...                           ...  ..   
 99995  09.12.20  10324808-30       0,00  Выполнение завершено успешно NaN   
 99996  09.12.20  10324808-30     345,29  Выполнение завершено успешно NaN   
 99997  09.12.20  10324808-30       0,00  Выполнение завершено успешно NaN   
 99998  09.12.20  10324808-30       0,00  Выполнение завершено успешно NaN   
 99999  09.12.20  10324858-30     417,34  Выполнение завершено успешно NaN   
 
                                                              

In [159]:
# Объединяем датафреймы и сбрасываем старые индексы и создаем новые
sbis = pd.concat(dfs, ignore_index=True)
# Присваиваем столбцам новые наименования
sbis.columns = sbis_names
# Все названия из нескольких слов сцепляем через нижнее подчеркивание в одно_длинное_слово
sbis.columns = [c.replace(' ', '_') for c in sbis.columns]
sbis.head(2)

,Дата,Номер,Сумма,Статус,Примечание,Комментарий,Контрагент,ИНН/КПП,Организация,ИНН/КПП,Тип_документа,Имя_файла,Дата,Номер_1,Сумма_1,Сумма_НДС,Ответственный,Подразделение,Код,Дата,Время,Тип_пакета,Идентификатор_пакета,Запущено_в_обработку,Получено_контрагентом,Завершено,Увеличение_суммы,НДC,Уменьшение_суммы,НДС
0,30.09.21,БРН00709545,"4 056,46",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,ЭДОНакл,DP_TOVTORGPR_2BE911035223f7011e2addf005056917125_2BE692ffbaa8a4711e39c98005056917125_20210930_75b0f1b8-222e-11ec-b9bc-00e0ed9e2e98.xml,30.09.21,БРН00709545,"4 056,46","368,77",NaN,Аптека №1,3,30.09.21,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,30.09.21 23:40,01.10.21 08:39,01.10.21 14:59,"0,00","0,00","0,00","0,00"
1,30.09.21,БРН00709545,"4 056,46",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,СчФктр,ON_NSCHFDOPPR_2BE911035223f7011e2addf005056917125_2BE692ffbaa8a4711e39c98005056917125_20210930_75b0f1b8-222e-11ec-b9bc-00e0ed9e2e98.xml,30.09.21,БРН00709545,"4 056,46","368,77",NaN,Аптека №1,3,30.09.21,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"


In [160]:
# Определяем путь нахождения файлов с выгрузкой из аптек — папка 'Аптеки\csv\correct\'
apteka_folder = r'C:\Users\1C\Аптеки\csv\correct'
# apteka_folder
apteka_files = os.listdir(apteka_folder)
apteka_files

['366.csv', 'А123.csv']

In [151]:
# Загружайте на обработку по одному файлу. Содержимое каждого файла загружайте в датафрейм Pandas. 
# Не csv-формат — игнорируем
for file in apteka_files:

    if 'csv' not in file:
        continue

    apteka = pd.read_csv(apteka_folder + '/' + file, sep = ";", encoding="1251")
    
    # Создаем список из новых столбцов
    cols = ["Номер счет-фактуры", "Сумма счет-фактуры", "Дата счет-фактуры", "Сравнение дат"]
    
    # 
    for col in cols:
        apteka[col] = ''
    
    # Создаем список из нужных типов документов
    docs = ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]

    # В каждой строке проверить:
    for i, row in apteka.iterrows():
        nakl = row["Номер накладной"]

        # Если "Поставщик" — ЕАПТЕКА, то к "Номер накладной" нужно добавить /15
        if 'ЕАПТЕКА' in row["Поставщик"]:
            nakl += "/15"
        
        # Нужно найти все записи в выгрузке из СБИСа по данному номеру накладной
        records = sbis[sbis.Номер.values == nakl]
        # Из найденных строк нужно оставить только те, которые имеют один из типов документа: ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]
        records = records[records.Тип_документа.isin(docs)]
        
        # Если ничего не найдено — просто переходим к следующей строке 
        if records.empty:
            continue
        
        # Если найдено — нужно сохранить значения Номер, Сумма и Дата
        # Дату нужно представить в формате 25.05.2021
        invoice = records.iloc[0]["Номер"]
        summ = records.iloc[0]["Сумма"]
        date = records.iloc[0]["Дата"][1]
        date = datetime.strptime(date, "%d.%m.%y").strftime("%d.%m.%Y")

        # В столбцы из пункта 6 нужно записать найденные для данной строки значения
        apteka.at[i, "Номер счет-фактуры"] = invoice
        apteka.at[i, "Сумма счет-фактуры"] = summ
        apteka.at[i, "Дата счет-фактуры"] = date
        # В столбец Сравнение дат помещаем "Не совпадает!", если найденная дата и дата накладной отличаются. 
        # Иначе — пустая строка
        apteka.at[i, "Сравнение дат"] = "" if (date == apteka.at[i, 'Дата накладной']) else "Не совпадает!"

    # Определяем порядок столбцов в итоговом файле выгрузки по аптеке
    apteka_columns = ['№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
        'Дата приходного документа', 'Номер приходного документа',
        'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
        'Сумма счет-фактуры', 'Кол-во',
        'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
        'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат']
    
    # Определяем путь сохранения файла
    apteka = apteka[apteka_columns]
    apteka.to_excel(f"{result_path}{file.split('.csv')[0]} - результат.xlsx", index=False, encoding="windows-1251")
    print(f'{file} Обработан!')

366.csv Обработан!
А123.csv Обработан!
